In [2]:
from data_prepare import prepare_embed_df, make_folds

In [3]:
df = prepare_embed_df()

In [4]:
train_folds, valid_folds = make_folds(df)

In [12]:
train_folds[0].label.value_counts()

1    28136
0    27761
Name: label, dtype: int64